**How do we measure the relative performance of our retrieval system if we change one of it's parameters...for example, changing our embedding model? What about measuring system performance over time as more documents are added to our datastore?**

**We need a way to evaluate our retrieval system, and this notebook will show you "one way" of doing that. Ultimately, measuring retrieval performance is hard because it requires a lot of time and effort, requires some form of data labeling. With the advent of powerful generative LLMs the process of measuring retrieval performance has become much easier.**

# Retrieval Evaluation - Process
***
Here's a high-level overview of how the Retrieval Evaluation process in this notebook works:

1. Generate a "golden dataset" of query-context pairs.  Used a pseudo-LlamaIndex implementation for this step. 100 document chunks (contexts) were randomly selected from the Impact Theory corpus and those chunks were then submitted to the `gpt-3.5-turbo` model which generated a query that could be answered by the context.  The output was 100 query-context pairs along with associated doc_ids. 
   - **Assumptions**:
     - The generated query-context pairs are, in fact, relevant to one another i.e. the query can be answered by the context that it's paired with
     - The generated queries are simliar in style and length to the type of queries that end users would ask
2. The golden dataset consists of three primary keys: `corpus`, `relevant_docs`, and `queries`
     - The `corpus` is the original text context/chunk with it's associated `doc_id`
     - The `queries` are the LLM generated queries, one (or more) for each entry in the `corpus`
     - The `relevant_docs` is a simple lookup table linking the `corpus` docs to the generated `queries`
3. We pass the golden dataset into a retrieval evluation function which does the following:
   - Takes in a `retriever` arg (`WeaviateClient`) and a few other configuration params
   - Iterates over all queries in the golden dataset and retrieves search results for each query from Weaviate datastore
   - Extracts all `doc_id` values from the retrieved results
   - Extracts the `doc_id` from the associated `relevant_docs` for each query
   - Checks if the relevant doc_id is in the list of retrieved result doc_ids
   - After all queries are completed a `hit_rate` score and `mrr` score are calculated for the entire golden dataset
   - Writes results to an `eval_results` folder

#### In a Nutshell
Ulitmately, given a golden dataset consisting of queries, relevant docs, and their associated doc_ids, the `retrieval_evaluation` function is checking if the relevant doc_id is found in the list of retrieved results doc_ids, for each query.

#### Problems with this Approach
The problems with this approach are many:
- The **Assumptions** (see section 1 above) about the golden dataset must hold true.  Given that the pairs are generated by `gpt-3.5-turbo`, I think the first assumption will generally be true.  The 2nd assumption though, is going to be dependent on your particular search use case.
- This approach is conservative in that there is only "one" right answer.  Either the relevant `doc_id` is in the results list or it isn't.  In reality, there are going to be several documents that could potentially answer the generated query, but we have no way to account for these other relevant documents, unless of course, we want to manually add doc_ids to the golden dataset (and depending on your business case, you may actually want to do that).
- We aren't measuring recall or precision because we aren't classifying other documents as "negatives".  As was just mentioned, the other documents in the results list may or may not be good matches, we just don't know.  Because we don't know, we can't really classify the other documents as "negatives".  So for this approach, we are measuring the 'hit rate' which is simply a count of the number of times that we found a relevant `doc_id` match in the results list and `Mean Reciprocal Rank (MRR)`.  We're using MRR over other metrics such as Mean Average Precision (MAP) because we are only looking at a [single relevant answer](https://stats.stackexchange.com/questions/127041/mean-average-precision-vs-mean-reciprocal-rank).  Hit rate is a good enough metric for determining if our retriever is retrieving quality results, and MRR will become more important later on when we add a Reranker to the mix.  

In [ ]:
#standard library imports
from typing import List, Tuple, Dict, Any
import time
import os

# utilities
from tqdm.notebook import tqdm
from rich import print
from dotenv import load_dotenv
env = load_dotenv('./.env', override=True)

In [ ]:
from retrieval_evaluation import calc_hit_rate_scores, calc_mrr_scores, record_results, add_params
from llama_index.finetuning import EmbeddingQAFinetuneDataset
from weaviate_interface import WeaviateClient

golden_dataset = EmbeddingQAFinetuneDataset.from_json("data/golden_100.json")
# should see 100 queries
print(f'Num queries in Golden Dataset: {len(golden_dataset.queries)}')


### Instantiate Weaviate client and set Class name
# read env vars from local .env file
api_key = ""
url = ""

client = WeaviateClient(api_key, url)
class_name = "Impact_theory_minilm_256"

#check if WCS instance is live and ready
client.is_live(), client.is_ready()

In [26]:
# print (list(golden_dataset.queries.items())[0:3])
# print (list(golden_dataset.corpus.items())[0:3])
# print (list(golden_dataset.relevant_docs.items())[0:3])

# Retrieval Evaluation

In [28]:
from typing import Dict, Union
def retrieval_evaluation(dataset: EmbeddingQAFinetuneDataset, 
                         class_name: str, 
                         retriever: WeaviateClient,
                         retrieve_limit: int=5,
                         chunk_size: int=256,
                         hnsw_config_keys: List[str]=['maxConnections', 'efConstruction', 'ef'],
                         display_properties: List[str]=['doc_id', 'guest', 'content'],
                         dir_outpath: str='./eval_results',
                         include_miss_info: bool=False,
                         user_def_params: Dict[str,Any]=None
                         ) -> Dict[str, Union[str, int, float]]:
    '''
    Given a dataset and a retriever evaluate the performance of the retriever. Returns a dict of kw and vector
    hit rates and mrr scores. If inlude_miss_info is True, will also return a list of kw and vector responses 
    and their associated queries that did not return a hit, for deeper analysis. 

    Args:
    -----
    dataset: EmbeddingQAFinetuneDataset
        Dataset to be used for evaluation
    class_name: str
        Name of Class on Weaviate host to be used for retrieval
    retriever: WeaviateClient
        WeaviateClient object to be used for retrieval 
    retrieve_limit: int=5
        Number of documents to retrieve from Weaviate host
    chunk_size: int=256
        Number of tokens used to chunk text. This value is purely for results 
        recording purposes and does not affect results. 
    display_properties: List[str]=['doc_id', 'content']
        List of properties to be returned from Weaviate host for display in response
    dir_outpath: str='./eval_results'
        Directory path for saving results.  Directory will be created if it does not
        already exist. 
    include_miss_info: bool=False
        Option to include queries and their associated kw and vector response values
        for queries that are "total misses"
    user_def_params : dict=None
        Option for user to pass in a dictionary of user-defined parameters and their values.
    '''

    results_dict = {'n':retrieve_limit, 
                    'Retriever': retriever.model_name_or_path, 
                    'chunk_size': chunk_size,
                    'kw_hit_rate': 0,
                    'kw_mrr': 0,
                    'vector_hit_rate': 0,
                    'vector_mrr': 0,
                    'total_misses': 0,
                    'total_questions':0
                    }
    #add hnsw configs and user defined params (if any)
    results_dict = add_params(client, class_name, results_dict, user_def_params, hnsw_config_keys)
    
    start = time.perf_counter()
    miss_info = []
    for query_id, q in tqdm(dataset.queries.items(), 'Queries'):
        results_dict['total_questions'] += 1
        hit = False
        #make Keyword, Vector, and Hybrid calls to Weaviate host
        try:
            kw_response = retriever.keyword_search(request=q, class_name=class_name, limit=retrieve_limit, display_properties=display_properties)
            vector_response = retriever.vector_search(request=q, class_name=class_name, limit=retrieve_limit, display_properties=display_properties)
            
            #collect doc_ids and position of doc_ids to check for document matches
            kw_doc_ids = {result['doc_id']:i for i, result in enumerate(kw_response, 1)}
            vector_doc_ids = {result['doc_id']:i for i, result in enumerate(vector_response, 1)}
            
            #extract doc_id for scoring purposes
            doc_id = dataset.relevant_docs[query_id][0]
     
            #increment hit_rate counters and mrr scores
            if doc_id in kw_doc_ids:
                results_dict['kw_hit_rate'] += 1
                results_dict['kw_mrr'] += 1/kw_doc_ids[doc_id]
                hit = True
            if doc_id in vector_doc_ids:
                results_dict['vector_hit_rate'] += 1
                results_dict['vector_mrr'] += 1/vector_doc_ids[doc_id]
                hit = True
                
            # if no hits, let's capture that
            if not hit:
                results_dict['total_misses'] += 1
                miss_info.append({'query': q, 'kw_response': kw_response, 'vector_response': vector_response})
        except Exception as e:
            print(e)
            continue
    

    #use raw counts to calculate final scores
    calc_hit_rate_scores(results_dict)
    calc_mrr_scores(results_dict)
    
    end = time.perf_counter() - start
    print(f'Total Processing Time: {round(end/60, 2)} minutes')
    record_results(results_dict, chunk_size, dir_outpath=dir_outpath, as_text=True)
    
    if include_miss_info:
        return results_dict, miss_info
    return results_dict

In [ ]:
results = retrieval_evaluation(golden_dataset, 
                         class_name, 
                         client,
                         5,
                         256,
                         ['maxConnections', 'efConstruction', 'ef'],
                         ['doc_id', 'guest', 'content'],
                         './eval_results',
                         False,
                         {'maxConnections':32, 'efConstruction':128, 'ef':-1}
                         )